In [62]:
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F
import os
from dotenv import load_dotenv
from pyspark.ml.feature import Imputer
import io
from google.cloud import storage

In [2]:
load_dotenv()

spark = SparkSession \
    .builder \
    .master('spark://localhost:7077') \
    .config("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.36.1") \
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
     .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
     .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
     .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
     .config("spark.hadoop.google.cloud.auth.type", "SERVICE_ACCOUNT_JSON_KEYFILE") \
     .config("spark.hadoop.fs.gs.project.id", os.getenv('PROJECT_ID')) \
     .appName("preprocessing_spark") \
     .getOrCreate()

:: loading settings :: url = jar:file:/home/Bagas/spark/spark-3.5.5-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/Bagas/.ivy2/cache
The jars for the packages stored in: /home/Bagas/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a7b8c5fd-1e76-462f-88e0-891facf138e1;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.36.1 in central
:: resolution report :: resolve 128ms :: artifacts dl 4ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.36.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	-------------------------------------------------------------------

In [8]:
spark

In [6]:
# Load Data
base_dir = "gs://project-abd/notebook-data/"

try:
    df_survey = spark.read.csv(os.path.join(base_dir, "participant_survey.csv"),
                                header=True,
                                inferSchema=True)
    
    df_devices = spark.read.csv(os.path.join(base_dir, "fisiology.csv"),
                                header=True,
                                inferSchema=True)
except Exception as e:
    print(f'Error reading data from GCS : {e}')

In [7]:
survey = df_survey
devices = df_devices

survey.show(5)
devices.show(5)

+----------+-----------+---+----+-------+-------------------+----------------+----+--------+
|student_id|gender_code|age|week|weekday|               time|engagement_level|room|class_id|
+----------+-----------+---+----+-------+-------------------+----------------+----+--------+
|        15|          1| 16|   1|      1|2025-06-02 11:00:00|  Highly Engaged|  R3|       2|
|        20|          1| 16|   1|      1|2025-06-02 11:00:00|  Highly Engaged|  R3|       2|
|        13|          1| 16|   1|      1|2025-06-02 11:00:00|         Engaged|  R3|       2|
|        23|          1| 16|   1|      1|2025-06-02 11:00:00|  Highly Engaged|  R3|       2|
|        21|          1| 15|   1|      1|2025-06-02 11:00:00|     Not Engaged|  R3|       2|
+----------+-----------+---+----+-------+-------------------+----------------+----+--------+
only showing top 5 rows



+--------+--------------+-------------------+------------------+------------------+--------------------+-------------------+--------------------+
|class_id|participant_id|        time_format|           hr_mean|         temp_mean|            eda_mean|           ibi_mean|            bvp_mean|
+--------+--------------+-------------------+------------------+------------------+--------------------+-------------------+--------------------+
|       1|             1|2025-06-02 09:20:00|101.96850751051262| 18.79666699303521|0.001454626731881...|0.46877134839693707|0.005812729376985872|
|       1|             1|2025-06-02 09:25:00|112.20290006001791| 18.79920024236043|3.533424931811168...|0.38022567331790924| 0.01111510058719432|
|       1|             1|2025-06-02 09:30:00|110.07446683247885|18.782200450897218|3.661524929339066E-4| 0.3854343295097351|-0.01442968695181...|
|       1|             1|2025-06-02 09:35:00| 124.7051665242513| 18.65323377609253|2.006899961270392E-4|0.38790275516668205|

In [15]:
survey = survey.withColumnRenamed('time','time_survey')
survey = survey.withColumnRenamed('class_id','class_id_survey')

In [17]:
df_join = devices.join(survey, on=[
    (devices['class_id'] == survey['class_id_survey']),
    (devices['participant_id'] == survey['student_id']),
    (devices['time_format'] == survey['time_survey']),
], how='inner')

df_join.show()

+--------+--------------+-------------------+------------------+------------------+-------------------+------------------+--------------------+----------+-----------+---+----+-------+-------------------+----------------+----+---------------+
|class_id|participant_id|        time_format|           hr_mean|         temp_mean|           eda_mean|          ibi_mean|            bvp_mean|student_id|gender_code|age|week|weekday|        time_survey|engagement_level|room|class_id_survey|
+--------+--------------+-------------------+------------------+------------------+-------------------+------------------+--------------------+----------+-----------+---+----+-------+-------------------+----------------+----+---------------+
|       5|             1|2025-06-02 14:20:00| 81.25466677347819| 32.09609999974569| 0.1823231962067075|0.6698896481144813| 0.12214216527655178|         1|          1| 16|   1|      1|2025-06-02 14:20:00|  Highly Engaged|  R3|              5|
|       7|             1|2025-06

In [33]:
schema = types.StructType([
    types.StructField("class_id", types.IntegerType(), True),
    types.StructField("room", types.StringType(), True),
    types.StructField("student_id", types.IntegerType(), True),
    types.StructField("gender_code", types.IntegerType(), True),
    types.StructField("age", types.IntegerType(), True),
    types.StructField("time_format", types.TimestampType(), True), 
    types.StructField("hr_mean", types.DoubleType(), True),
    types.StructField("temp_mean", types.DoubleType(), True),
    types.StructField("eda_mean", types.DoubleType(), True),
    types.StructField("ibi_mean", types.DoubleType(), True),     
    types.StructField("bvp_mean", types.DoubleType(), True),
    types.StructField("engagement_level", types.StringType(), True)
])

for field in schema.fields:
     df_clean = df_clean.withColumn(field.name, F.col(f"`{field.name}`").cast(field.dataType))

In [34]:
df_clean.printSchema()

root
 |-- class_id: integer (nullable = true)
 |-- room: string (nullable = true)
 |-- student_id: integer (nullable = true)
 |-- gender_code: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- time_format: timestamp (nullable = true)
 |-- hr_mean: double (nullable = true)
 |-- temp_mean: double (nullable = true)
 |-- eda_mean: double (nullable = true)
 |-- ibi_mean: double (nullable = true)
 |-- bvp_mean: double (nullable = true)
 |-- engagement_level: string (nullable = true)



In [29]:
df_clean.show(5)

+--------+----+----------+-----------+---+-------------------+-----------------+------------------+-------------------+------------------+--------------------+----------------+
|class_id|room|student_id|gender_code|age|        time_format|          hr_mean|         temp_mean|           eda_mean|          ibi_mean|            bvp_mean|engagement_level|
+--------+----+----------+-----------+---+-------------------+-----------------+------------------+-------------------+------------------+--------------------+----------------+
|       5|  R3|         1|          1| 16|2025-06-02 14:20:00|81.25466677347819| 32.09609999974569| 0.1823231962067075|0.6698896481144813| 0.12214216527655178|  Highly Engaged|
|       7|  R3|         1|          1| 16|2025-06-02 10:55:00|83.31206672668458|30.882199986775717|0.06692810590456551|0.8627627087490899|-0.13621666791257062|  Highly Engaged|
|       7|  R3|        12|          1| 16|2025-06-02 10:55:00|84.71183344523112|31.020599892934165| 0.3389558720837

In [28]:
null_value = df_clean.select([F.sum(F.col(cols).isNull().cast('int')).alias(cols) for cols in df_clean.columns])
null_value.show()

+--------+----+----------+-----------+---+-----------+-------+---------+--------+--------+--------+----------------+
|class_id|room|student_id|gender_code|age|time_format|hr_mean|temp_mean|eda_mean|ibi_mean|bvp_mean|engagement_level|
+--------+----+----------+-----------+---+-----------+-------+---------+--------+--------+--------+----------------+
|       0|   0|         0|          0|  0|          0|      0|        0|       2|      52|       0|               0|
+--------+----+----------+-----------+---+-----------+-------+---------+--------+--------+--------+----------------+



In [56]:
imputer = Imputer(inputCols=['eda_mean','ibi_mean'],
                  outputCols=['eda_mean_inputed','ibi_mean_inputed'],
                  strategy='mean')
df = imputer.fit(df_clean).transform(df_clean)

df = df.drop('ibi_mean','eda_mean')
df = df.withColumnRenamed('eda_mean_inputed','eda_mean') \
        .withColumnRenamed('ibi_mean_inputed','ibi_mean')

In [57]:
null_value = df.select([F.sum(F.col(cols).isNull().cast('int')).alias(cols) for cols in df.columns])
null_value.show()

+--------+----+----------+-----------+---+-----------+-------+---------+--------+----------------+--------+--------+
|class_id|room|student_id|gender_code|age|time_format|hr_mean|temp_mean|bvp_mean|engagement_level|eda_mean|ibi_mean|
+--------+----+----------+-----------+---+-----------+-------+---------+--------+----------------+--------+--------+
|       0|   0|         0|          0|  0|          0|      0|        0|       0|               0|       0|       0|
+--------+----+----------+-----------+---+-----------+-------+---------+--------+----------------+--------+--------+



In [58]:
df = df.distinct()

In [59]:
df = df.withColumn('timestamp', F.date_format('time_format', 'HH:mm:ss')).drop('time_format')
df.show(5)

+--------+----+----------+-----------+---+-----------------+------------------+--------------------+----------------+-------------------+------------------+---------+
|class_id|room|student_id|gender_code|age|          hr_mean|         temp_mean|            bvp_mean|engagement_level|           eda_mean|          ibi_mean|timestamp|
+--------+----+----------+-----------+---+-----------------+------------------+--------------------+----------------+-------------------+------------------+---------+
|       7|  R3|        21|          1| 15|96.89173352559408|29.075799973805747| 0.14733649224615267|  Highly Engaged| 0.2409935087157646|0.7601556011424158| 10:55:00|
|     202|  R1|        20|          1| 16|84.53866225022536|   32.529366131322|0.024287718439838422|  Highly Engaged|0.15323040743138613|0.7591810502657076| 14:15:00|
|      78|  R3|         1|          1| 16|71.10822733748319| 32.39248572278758|-0.03438250654092...|     Not Engaged|0.16132125217433368|0.8025367164611816| 11:25:00

In [60]:
df = df.select('class_id','room','student_id','gender_code','age','timestamp','hr_mean','temp_mean','eda_mean','ibi_mean','bvp_mean','engagement_level')

In [63]:
df_pd = df.toPandas()
csv_buffer = io.StringIO()
df_pd.to_csv(csv_buffer, index=False)
csv_data = csv_buffer.getvalue()
csv_buffer.close()

In [68]:
client = storage.Client()
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/home/Bagas/project/credentials/credential.json'

In [69]:
bucket = client.get_bucket('project-abd')
destination_file = 'notebook-data/fusion_data.csv'
blob_csv = bucket.blob(destination_file)
blob_csv.upload_from_string(csv_data, content_type='text/csv')
print(f"DataFrame uploaded to gs://{bucket}/{destination_file} as CSV")

DataFrame uploaded to gs://<Bucket: project-abd>/notebook-data/fusion_data.csv as CSV
